In [70]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
from src import filter_assets
from src import plot
from src import portfolio
from src import capm
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
import datetime
from dateutil import rrule

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\Tor Osted\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['rrule', 'plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [71]:
x = pd.Timestamp(np.datetime64('2015-01-01'))
x1 = pd.Timestamp(np.datetime64('2022-01-01'))

In [72]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1400
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")
dates = [x,x1]

start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01
covariance_window = 5 #has to be in years



esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type,start_year,end_year, Wanted_return, maximum_risk,'yearly') #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.
sp500 = data.data_for_beta(np.append(['SPY'],(weights_of_each_portfolio.columns)),dates)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio)
capm_expected_return = capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)





c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 20.680% => Return: 33.506%  Sharpe Ratio = 1.62
Max. Sharpe Ratio = 1.63 => Return: 33.84%  Risk: 20.784%
Excpected return on investment is 100.09707103820338%


In [74]:
# import packages
from datetime import date
from dateutil.relativedelta import relativedelta


def backtesting(strategy,monthly_or_yearly_rebalancing,rebalancing_freq,start_date,end_date,covariance_window):
    #start = 2015
   # end = 2022

    covariance_window_time_delta = relativedelta(years=covariance_window) #The time delta for the covariance window
    print([start_date-covariance_window_time_delta,end_date])
    esg_data = data.esg_score_weight(strategy[0],strategy[1],strategy[2],strategy[3])
    full_data = data.stock_monthly_close(esg_data,[pd.Timestamp(start_date-covariance_window_time_delta),pd.Timestamp(end_date)])
    prices,esgdata = data.seperate_full_data(full_data)
    pct_returns = data.pct_returns_from_prices(prices)

    
    listparameters = []
    list_of_port_weights =[]
    list_of_port_esg_scores = []
    list_of_port_allocations = []
    list_of_cmle_returns = []
    sp500_list = []
    list_of_pct_returns_sp500 = []
    betas_of_portfolios = []
    i = 0
    if monthly_or_yearly_rebalancing == 'yearly':
        delta = relativedelta(years=rebalancing_freq)
        
        while (start_date <= end_date):
            print(start_date-covariance_window_time_delta,start_date)
            
            

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy[10])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            start_date += delta

        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,list_of_cmle_returns)

    elif monthly_or_yearly_rebalancing == 'monthly':
        delta = delta = np.timedelta64(rebalancing_freq,'M')
        while (start_date <= end_date):#The covariance window is given in years

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy[10])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            start_date += delta
            end_date += delta
            i += 1
        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,list_of_cmle_returns)

    
    else:
        print('You can only call this function with monthly or yearly')
        return(False)
    

backtesting(strategy1,'yearly',1,datetime.datetime(2015,1,1),datetime.datetime(2022,1,1),10)
#backtesting(strategy1,'yearly',1,np.datetime64('2015-01-01'),np.datetime64('2022-01-01'),10) #US dates YYYY-MM-DD

strategy1 = [esg_df,
             weights,
             min_esg_score,
             max_esg_score,
             Bounds1,
             sharpe_type,
             start,
             end,
             Wanted_return,
             maximum_risk,
             'monthly']

#This makes each portfolio using strategy 1 rebalancing every 6 months from 2015 until 2022, using a covariance window of ten years

#This makes each portfolio for 


[datetime.datetime(2005, 1, 1, 0, 0), datetime.datetime(2022, 1, 1, 0, 0)]
2005-01-01 00:00:00 2015-01-01 00:00:00
Min. Risk = 6.540% => Return: 0.888%  Sharpe Ratio = 0.14
Max. Sharpe Ratio = 0.21 => Return: 2.10%  Risk: 10.057%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


2006-01-01 00:00:00 2016-01-01 00:00:00
Min. Risk = 6.876% => Return: 1.230%  Sharpe Ratio = 0.18
Max. Sharpe Ratio = 0.25 => Return: 2.31%  Risk: 9.429%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


2007-01-01 00:00:00 2017-01-01 00:00:00
Min. Risk = 6.818% => Return: 1.390%  Sharpe Ratio = 0.20
Max. Sharpe Ratio = 0.29 => Return: 2.84%  Risk: 9.731%


...

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


2011-01-01 00:00:00 2021-01-01 00:00:00
Min. Risk = 5.577% => Return: 2.100%  Sharpe Ratio = 0.38
Max. Sharpe Ratio = 0.39 => Return: 2.27%  Risk: 5.795%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


2012-01-01 00:00:00 2022-01-01 00:00:00
Min. Risk = 5.654% => Return: 2.439%  Sharpe Ratio = 0.43
Max. Sharpe Ratio = 0.44 => Return: 2.49%  Risk: 5.715%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


In [78]:
#5.807% ,5.712%

portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type, datetime.datetime(2012,1,1),datetime.datetime(2022,1,1), Wanted_return,maximum_risk,'monthly')

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  bounds=bounds,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  x0,


Min. Risk = 5.970% => Return: 2.793%  Sharpe Ratio = 0.47
Max. Sharpe Ratio = 0.47 => Return: 2.82%  Risk: 6.000%


[(0.06000025165760809,
  0.02820449467967918,
  0.05969877658433012,
  0.027929813607204694,
  array([0.05972545, 0.05972602, 0.05972662, 0.05972727, 0.05972796,
         0.0597287 , 0.05972949, 0.05973034, 0.05973126, 0.05973225,
         0.05973332, 0.05973449, 0.05973576, 0.05973715, 0.05973867,
         0.05974035, 0.05974221, 0.05974427, 0.05974658, 0.05974917,
         0.05975209, 0.05975542, 0.05975923, 0.05976363, 0.05976876,
         0.05977481, 0.05978201, 0.0597907 , 0.05980135, 0.05981465,
         0.05983159, 0.05985371, 0.05988347, 0.05992502, 0.05998573,
         0.06007982, 0.06023731, 0.06052991, 0.0924617 , 0.0924617 ,
         0.0924617 , 0.0924617 , 0.07036798, 0.13360483, 0.13360483,
         0.13360483, 0.13360483, 0.13360483, 0.13360483, 0.06654754]),
  array([-0.3       , -0.29215075, -0.28430149, -0.27645224, -0.26860298,
         -0.26075373, -0.25290447, -0.24505522, -0.23720596, -0.22935671,
         -0.22150745, -0.2136582 , -0.20580894, -0.19795969, -0.190